# Initialize AI Search

This sample demonstrates how to create an index for Azure AI Search using semantic tagging and advanced filtering capabilities.

## Environment Variables Configuration

Update your `.env` file with the following variables for Document Intelligence and AI Search:

```bash
# Azure AI Search
AZURE_AI_SEARCH_ENDPOINT=https://your-search-service.search.windows.net
AZURE_AI_SEARCH_API_KEY=your-search-api-key
AZURE_SEARCH_INDEX_NAME=document_inquiry_index

# Azure OpenAI
AZURE_OPENAI_ENDPOINT=https://your-openai-service.openai.azure.com
AZURE_OPENAI_API_KEY=your-openai-api-key
AZURE_OPENAI_DEPLOYMENT_NAME=gpt-4o
AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME=text-embedding-3-small
AZURE_OPENAI_EMBEDDING_DIMENSIONS=1536
AZURE_OPENAI_API_VERSION=2024-12-01-preview

# Azure Document Intelligence
AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT=https://your-document-intelligence.cognitiveservices.azure.com
AZURE_DOCUMENT_INTELLIGENCE_API_KEY=your-document-intelligence-api-key

```

> ✨ **_Note_** <br>
> **Check Region support**: Please check the regional support for Azure AI Search before you get started - https://learn.microsoft.com/en-us/azure/search/search-region-support <br>
> **Pricing tier for Semantic Search**: In order to use the Semantic Search feature, check your region availability and pricing tier. Make sure it is at least Standard S3. <br>
> **Collection field limitation**: fuzzy search is not supported for collection fields.

## Key Features:
- **doc_upload_plugin**: Enables document upload and indexing into AI Search
- **ai_search_plugin**: Search capabilities with semantic ranking


Create an `.env` file based on the `sample.env` file. Copy the new `.env` file to the folder containing your notebook and update the variables.

In [2]:
import os
import json
import requests
import time
import subprocess
import sys
import json
from openai import AzureOpenAI
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents import SearchClient
from azure.search.documents import SearchIndexingBufferedSender
from azure.search.documents.models import (
    VectorizedQuery,
    VectorizableTextQuery,
    VectorFilterMode,
)
from azure.search.documents.models import QueryType, QueryCaptionType, QueryAnswerType
from azure.search.documents.indexes.models import (
    SimpleField,
    SearchFieldDataType,
    SearchableField,
    SearchField,
    HnswAlgorithmConfiguration,
    VectorSearch,
    VectorSearchProfile,
    SemanticConfiguration,
    SemanticPrioritizedFields,
    SemanticField,
    SemanticSearch,
    SearchIndex,
    AzureOpenAIVectorizer,
    AzureOpenAIVectorizerParameters,
)

# Load environment variables
load_dotenv(override=True)


# Validate environment variables
def get_env_var(name, required=True):
    value = os.getenv(name)
    if required and not value:
        raise ValueError(f"Environment variable {name} is missing or empty.")
    return value


try:
    azure_ai_search_endpoint = os.getenv("AZURE_AI_SEARCH_ENDPOINT")
    azure_ai_search_admin_key = get_env_var("AZURE_AI_SEARCH_API_KEY", required=True)

    azure_openai_endpoint = get_env_var("AZURE_OPENAI_ENDPOINT")
    azure_openai_key = get_env_var("AZURE_OPENAI_API_KEY")
    azure_openai_deployment_name = get_env_var("AZURE_OPENAI_DEPLOYMENT_NAME")
    azure_openai_embedding_deployment_name = (
        get_env_var("AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME", required=False)
        or "text-embedding-3-small"
    )
    azure_openai_embedding_dimensions = int(
        get_env_var("AZURE_OPENAI_EMBEDDING_DIMENSIONS", required=False) or 1536
    )
    azure_openai_api_version = (
        get_env_var("AZURE_OPENAI_API_VERSION", required=False) or "2024-12-01-preview"
    )

    index_name = (
        get_env_var("AZURE_AI_SEARCH_INDEX_NAME", required=False)
        or "doc_inquiry_index"
    )

    print("==== Environment Variables ====")
    print(f"azure_search_endpoint={azure_ai_search_endpoint}")
    
    print(f"azure_openai_endpoint={azure_openai_endpoint}")
    
    print(f"azure_openai_deployment_name={azure_openai_deployment_name}")
    print(
        f"azure_openai_embedding_deployment_name={azure_openai_embedding_deployment_name}"
    )
    print(f"azure_openai_embedding_dimensions={azure_openai_embedding_dimensions}")
    print(f"index_name={index_name}")

    # Validate credentials
    if azure_ai_search_admin_key:
        search_credential = AzureKeyCredential(azure_ai_search_admin_key)
    else:
        search_credential = DefaultAzureCredential()
        # Check if DefaultAzureCredential works
        try:
            search_credential.get_token("https://management.azure.com/.default")
        except Exception as e:
            raise ValueError(
                "DefaultAzureCredential authentication failed. Ensure you are logged in using `az login`."
            ) from e

    print("All environment variables are valid.")

except ValueError as e:
    print(f"[ERROR] {e}")
    exit(1)

==== Environment Variables ====
azure_search_endpoint=https://hyo-rag-ai-search.search.windows.net
azure_openai_endpoint=https://hyo-ai-foundry-pjt1-resource.openai.azure.com/
azure_openai_deployment_name=gpt-4.1
azure_openai_embedding_deployment_name=text-embedding-3-small
azure_openai_embedding_dimensions=1536
index_name=doc_inquiry_index
All environment variables are valid.


In [3]:
if azure_openai_endpoint is None:
    raise ValueError("The Azure OpenAI endpoint is not set.")

openai_client = AzureOpenAI(
    api_version=azure_openai_api_version,
    azure_endpoint=azure_openai_endpoint,
    api_key=azure_openai_key,
)

### Create vector index (Execute Once)



In [4]:
index_client = SearchIndexClient(
    endpoint=azure_ai_search_endpoint, credential=search_credential
)

search_client = SearchClient(
    endpoint=azure_ai_search_endpoint,
    index_name=index_name,
    credential=search_credential,
)

In [9]:
from azure.core.exceptions import ResourceNotFoundError

CREATE_INDEX = True

if CREATE_INDEX:
    
    try:
        index_client.get_index(index_name)
        index_client.delete_index(index_name)
        print(f"Index '{index_name}' deleted.")
    except Exception as e:
        print(f"Index '{index_name}' does not exist, creating a new one.")
        print(f"Error details: {e}")
    
    # Enhanced document index fields for PDF/IR document processing
    fields = [
        # Primary identification fields
        SimpleField(name="docId", type=SearchFieldDataType.String, key=True),
        SearchableField(name="docName", type=SearchFieldDataType.String, sortable=True, filterable=True),
        SearchableField(name="fileName", type=SearchFieldDataType.String, sortable=True, filterable=True),
        
        # Document metadata
        SimpleField(name="fileSize", type=SearchFieldDataType.Int64, filterable=True, sortable=True),
        SimpleField(name="uploadDate", type=SearchFieldDataType.DateTimeOffset, filterable=True, sortable=True),
        SimpleField(name="lastModified", type=SearchFieldDataType.DateTimeOffset, filterable=True, sortable=True),
        SearchableField(name="fileType", type=SearchFieldDataType.String, filterable=True, facetable=True),
        
        # Content fields with language support
        SearchableField(
            name="content", 
            type=SearchFieldDataType.String, 
            analyzer_name="en.lucene"
        ),
        SearchableField(
            name="contentKo",
            type=SearchFieldDataType.String,
            analyzer_name="ko.microsoft",
        ),
        
        # Document categorization for IR/market research documents
        SearchableField(
            name="documentType",
            type=SearchFieldDataType.String,
            facetable=True,
            filterable=True,
            sortable=True,
        ),
        SearchableField(
            name="industry",
            type=SearchFieldDataType.String,
            facetable=True,
            filterable=True,
            sortable=True,
        ),
        SearchableField(
            name="company",
            type=SearchFieldDataType.String,
            facetable=True,
            filterable=True,
            sortable=True,
        ),
        SearchableField(
            name="reportYear",
            type=SearchFieldDataType.String,
            facetable=True,
            filterable=True,
            sortable=True,
        ),
        
        # Document sections and chunks for better retrieval
        SearchableField(name="summary", type=SearchFieldDataType.String),
        SearchableField(name="keywords", type=SearchFieldDataType.String),
        SimpleField(name="pageCount", type=SearchFieldDataType.Int32, filterable=True, sortable=True),
        SearchableField(name="author", type=SearchFieldDataType.String, filterable=True),
        
        # Source and URL information
        SearchableField(name="sourcePath", type=SearchFieldDataType.String),
        
        # Vector fields for semantic search
        SearchField(
            name="contentVector",
            type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
            searchable=True,
            vector_search_dimensions=azure_openai_embedding_dimensions,
            vector_search_profile_name="content-vector-profile",
        ),
        SearchField(
            name="summaryVector",
            type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
            searchable=True,
            vector_search_dimensions=azure_openai_embedding_dimensions,
            vector_search_profile_name="summary-vector-profile",
        ),
        
        # Document tags for advanced filtering
        SearchField(
            name="documentTags",
            type=SearchFieldDataType.Collection(SearchFieldDataType.String),
            searchable=True,
            filterable=True,
            facetable=True,
        ),
        SearchField(
            name="targetAudience",
            type=SearchFieldDataType.Collection(SearchFieldDataType.String),
            searchable=True,
            filterable=True,
            facetable=True,
        ),
        SearchField(
            name="topics",
            type=SearchFieldDataType.Collection(SearchFieldDataType.String),
            searchable=True,
            filterable=True,
            facetable=True,
        ),
    ]

    # Vector search configuration
    vector_search = VectorSearch(
        profiles=[
            VectorSearchProfile(
                name="content-vector-profile",
                algorithm_configuration_name="content-hnsw-config",
                vectorizer_name="content-openai-vectorizer",
            ),
            VectorSearchProfile(
                name="summary-vector-profile",
                algorithm_configuration_name="summary-hnsw-config",
                vectorizer_name="summary-openai-vectorizer",
            ),
        ],
        algorithms=[
            HnswAlgorithmConfiguration(
                name="content-hnsw-config",
                parameters={
                    "m": 4,
                    "efConstruction": 400,
                    "efSearch": 500,
                    "metric": "cosine",
                },
            ),
            HnswAlgorithmConfiguration(
                name="summary-hnsw-config",
                parameters={
                    "m": 4,
                    "efConstruction": 400,
                    "efSearch": 500,
                    "metric": "cosine",
                },
            ),
        ],
        vectorizers=[
            AzureOpenAIVectorizer(
                vectorizer_name="content-openai-vectorizer",
                parameters=AzureOpenAIVectorizerParameters(
                    resource_url=azure_openai_endpoint,
                    deployment_name=azure_openai_embedding_deployment_name,
                    model_name=azure_openai_embedding_deployment_name,
                    api_key=azure_openai_key,
                ),
            ),
            AzureOpenAIVectorizer(
                vectorizer_name="summary-openai-vectorizer",
                parameters=AzureOpenAIVectorizerParameters(
                    resource_url=azure_openai_endpoint,
                    deployment_name=azure_openai_embedding_deployment_name,
                    model_name=azure_openai_embedding_deployment_name,
                    api_key=azure_openai_key,
                ),
            ),
        ],
    )

    # Semantic search configuration for document retrieval
    semantic_config = SemanticConfiguration(
        name="document-semantic-config",
        prioritized_fields=SemanticPrioritizedFields(
            title_field=SemanticField(field_name="docName"),
            content_fields=[
                SemanticField(field_name="content"),
                SemanticField(field_name="summary"),
            ],
            keywords_fields=[
                SemanticField(field_name="keywords"),
                SemanticField(field_name="documentTags"),
                SemanticField(field_name="topics"),
            ],
        ),
    )

    semantic_search = SemanticSearch(configurations=[semantic_config])

    # Create the search index
    index = SearchIndex(
        name=index_name,
        fields=fields,
        vector_search=vector_search,
        semantic_search=semantic_search,
    )

    result = index_client.create_or_update_index(index)
    print(f"Index '{result.name}' created or updated successfully.")
else:
    print("Index creation is disabled. Set CREATE_INDEX = True to create the index.")

Index 'doc_inquiry_index' deleted.
Index 'doc_inquiry_index' created or updated successfully.


### Document Intelligence PDF Upload Plugin

Create upload_doc_plugin to process PDF files using Document Intelligence and upload to Azure AI Search.

In [10]:
# Document Intelligence setup
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.documentintelligence.models import AnalyzeDocumentRequest, DocumentContentFormat
import hashlib
import datetime
from typing import List, Dict, Any, Optional

# Document Intelligence environment variables
document_intelligence_endpoint = get_env_var("AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT")
document_intelligence_key = get_env_var("AZURE_DOCUMENT_INTELLIGENCE_API_KEY")

# Initialize Document Intelligence client
doc_intelligence_client = DocumentIntelligenceClient(
    endpoint=document_intelligence_endpoint,
    credential=AzureKeyCredential(document_intelligence_key)
)

print(f"Document Intelligence endpoint: {document_intelligence_endpoint}")
print("Document Intelligence client initialized successfully.")

Document Intelligence endpoint: https://hyo-ai-foundry-pjt1-resource.cognitiveservices.azure.com/
Document Intelligence client initialized successfully.


In [11]:
def upload_doc_plugin(
    file_path: str,
    doc_name: Optional[str] = None,
    document_type: str = "IR_REPORT",
    industry: Optional[str] = None,
    company: Optional[str] = None,
    report_year: Optional[str] = None,
    author: Optional[str] = None
) -> Dict[str, Any]:
    """
    Upload and process PDF document using Document Intelligence and store in Azure AI Search.
    
    Args:
        file_path: Path to the PDF file
        doc_name: Document name (defaults to filename)
        document_type: Type of document (IR_REPORT, MARKET_RESEARCH, etc.)
        industry: Industry category
        company: Company name
        report_year: Year of the report
        author: Document author
    
    Returns:
        Dict containing upload status and document metadata
    """
    try:
        # Generate document ID
        file_name = os.path.basename(file_path)
        doc_id = hashlib.md5(file_path.encode()).hexdigest()
        
        # Check if document already exists in the index
        try:
            existing_doc = search_client.get_document(key=doc_id)
            return {
                "status": "already_exists",
                "doc_id": doc_id,
                "message": f"Document {file_name} already exists in the index",
                "existing_doc": existing_doc
            }
        except Exception:
            # Document doesn't exist, proceed with upload
            pass
        
        print(f"Processing document: {file_name}")
        
        # Step 1: Process PDF with Document Intelligence
        with open(file_path, "rb") as file:
            poller = doc_intelligence_client.begin_analyze_document(
                model_id="prebuilt-layout",
                body=file, 
                output_content_format=DocumentContentFormat.MARKDOWN
            )
        
        result = poller.result()
        
        # Extract content and metadata
        content = result.content if result.content else ""
        # Extract metadata from document analysis
        page_count = len(result.pages) if result.pages else 0
        
        # TODO Consider page level chunking (can be extended by length/token if needed)
        # chunks = []
        # for p in result.paragraphs or []:
        #     page = p.bounding_regions[0].page_number if p.bounding_regions else None
        #     chunks.append({
        #         "page": page,
        #         "content": p.content,
        #         "offset": p.spans[0].offset,
        #         "length": p.spans[0].length
        #     })
        # #
        
        # Step 2: Generate embeddings for content and summary
        # Create summary from first 2000 characters of content
        summary = content[:2000] + "..." if len(content) > 2000 else content
        
        # Generate content embedding
        content_embedding_response = openai_client.embeddings.create(
            input=content[:8000],  # Limit content for embedding
            model=azure_openai_embedding_deployment_name
        )
        content_vector = content_embedding_response.data[0].embedding
        
        # Generate summary embedding
        summary_embedding_response = openai_client.embeddings.create(
            input=summary,
            model=azure_openai_embedding_deployment_name
        )
        summary_vector = summary_embedding_response.data[0].embedding
        
        # Step 3: Extract keywords and topics from content
        keywords = extract_keywords_from_content(content)
        
        # TODO Step 4: Generate topics, document tags, target audience using Azure OpenAI
        target_audience=["investors", "analysts", "stakeholders"]
        topics=["financial_performance", "quarterly_results", "revenue"]
        document_tags=["earnings", "Q1", "financial"]
        
        # TODO Step 5: translate the content in Korean
        content_ko = content  # Placeholder for translated content
        
        
        # Step 4: Prepare document for upload
        file_stats = os.stat(file_path)
        current_time = datetime.datetime.now(datetime.timezone.utc)
        
        document = {
            "docId": doc_id,
            "docName": doc_name or os.path.splitext(file_name)[0],
            "fileName": file_name,
            "fileSize": file_stats.st_size,
            "uploadDate": current_time.isoformat(),
            "lastModified": datetime.datetime.fromtimestamp(file_stats.st_mtime, datetime.timezone.utc).isoformat(),
            "fileType": "PDF",
            "content": content,
            "contentKo": content_ko,  
            "documentType": document_type,
            "industry": industry or "Unknown",
            "company": company or "Unknown",
            "reportYear": report_year or str(current_time.year),
            "summary": summary,
            "keywords": keywords,
            "pageCount": page_count,
            "author": author or "Unknown",
            "sourcePath": file_path,
            "contentVector": content_vector,
            "summaryVector": summary_vector,
            "documentTags": document_tags or [],
            "targetAudience": target_audience or [],
            "topics": topics or [],
        }
        
        with open("document.json", "w", encoding="utf-8") as f:
            json.dump(document, f, ensure_ascii=False, indent=2)
        
        # Step 5: Upload to Azure AI Search
        search_client.upload_documents([document])
        
        print(f"✅ Document {file_name} uploaded successfully!")
        
        return {
            "status": "success",
            "doc_id": doc_id,
            "doc_name": document["docName"],
            "message": f"Document {file_name} processed and uploaded successfully",
            "page_count": page_count,
            "content_length": len(content)
        }
        
    except Exception as e:
        print(f"❌ Error processing document {file_path}: {str(e)}")
        return {
            "status": "error",
            "message": f"Error processing document: {str(e)}"
        }


def extract_keywords_from_content(content: str, max_keywords: int = 20) -> str:
    """
    TODO In production, you might want to use gpt models.
    Extract keywords from document content using simple text processing.
    """
    import re
    from collections import Counter
    
    # Simple keyword extraction
    # Remove common stop words and extract meaningful terms
    stop_words = {
        'the', 'a', 'an', 'and', 'or', 'but', 'in', 'on', 'at', 'to', 'for', 'of', 'with', 'by',
        'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'do', 'does', 'did',
        'will', 'would', 'could', 'should', 'may', 'might', 'can', 'this', 'that', 'these', 'those'
    }
    
    # Extract words (letters only, minimum 3 characters)
    words = re.findall(r'\b[a-zA-Z]{3,}\b', content.lower())
    
    # Filter out stop words and count frequency
    meaningful_words = [word for word in words if word not in stop_words]
    word_counts = Counter(meaningful_words)
    
    # Get top keywords
    top_keywords = [word for word, _ in word_counts.most_common(max_keywords)]
    
    return ", ".join(top_keywords)



### AI Search Plugin for Document Retrieval

Create ai_search_plugin to perform hybrid and semantic search on uploaded documents.

In [12]:
def ai_search_plugin(
    query: str,
    search_type: str = "hybrid",  # "hybrid", "semantic", "vector", "text"
    filters: Optional[str] = None,
    top_k: int = 5,
    include_content: bool = True,
    document_type: Optional[str] = None,
    industry: Optional[str] = None,
    company: Optional[str] = None,
    report_year: Optional[str] = None
) -> Dict[str, Any]:
    """
    Search documents in Azure AI Search using various search methods.
    
    Args:
        query: Search query
        search_type: Type of search (hybrid, semantic, vector, text)
        filters: OData filter expression
        top_k: Number of results to return
        include_content: Whether to include full content in results
        document_type: Filter by document type
        industry: Filter by industry
        company: Filter by company
        report_year: Filter by report year
    
    Returns:
        Dict containing search results and metadata
    """
    try:
        # Generate query embedding for vector search
        query_embedding_response = openai_client.embeddings.create(
            input=query,
            model=azure_openai_embedding_deployment_name
        )
        query_vector = query_embedding_response.data[0].embedding
        
        # Build filter expression
        filter_parts = []
        if filters:
            filter_parts.append(filters)
        if document_type:
            filter_parts.append(f"documentType eq '{document_type}'")
        if industry:
            filter_parts.append(f"industry eq '{industry}'")
        if company:
            filter_parts.append(f"company eq '{company}'")
        if report_year:
            filter_parts.append(f"reportYear eq '{report_year}'")
        
        filter_expression = " and ".join(filter_parts) if filter_parts else None
        
        # Configure search based on search type
        search_results = None
        
        if search_type == "hybrid":
            # Hybrid search: combines text and vector search
            vector_queries = [
                VectorizedQuery(
                    vector=query_vector,
                    k_nearest_neighbors=top_k * 2,  # Get more for reranking
                    fields="contentVector"
                ),
                VectorizedQuery(
                    vector=query_vector,
                    k_nearest_neighbors=top_k,
                    fields="summaryVector"
                )
            ]
            
            search_results = search_client.search(
                search_text=query,
                vector_queries=vector_queries,
                filter=filter_expression,
                query_type=QueryType.SEMANTIC,
                semantic_configuration_name="document-semantic-config",
                query_caption=QueryCaptionType.EXTRACTIVE,
                query_answer=QueryAnswerType.EXTRACTIVE,
                top=top_k,
                select="docId,docName,fileName,content,summary,documentType,industry,company,reportYear,author,pageCount,uploadDate,keywords,documentTags,topics,targetAudience" if include_content else "docId,docName,fileName,summary,documentType,industry,company,reportYear,author,pageCount,uploadDate"
            )
            
        elif search_type == "semantic":
            # Semantic search only
            search_results = search_client.search(
                search_text=query,
                filter=filter_expression,
                query_type=QueryType.SEMANTIC,
                semantic_configuration_name="document-semantic-config",
                query_caption=QueryCaptionType.EXTRACTIVE,
                query_answer=QueryAnswerType.EXTRACTIVE,
                top=top_k,
                select="docId,docName,fileName,content,summary,documentType,industry,company,reportYear,author,pageCount,uploadDate,keywords,documentTags,topics,targetAudience" if include_content else "docId,docName,fileName,summary,documentType,industry,company,reportYear,author,pageCount,uploadDate"
            )
            
        elif search_type == "vector":
            # Vector search only
            vector_queries = [
                VectorizedQuery(
                    vector=query_vector,
                    k_nearest_neighbors=top_k,
                    fields="contentVector"
                )
            ]
            
            search_results = search_client.search(
                search_text=None,
                vector_queries=vector_queries,
                filter=filter_expression,
                top=top_k,
                select="docId,docName,fileName,content,summary,documentType,industry,company,reportYear,author,pageCount,uploadDate,keywords,documentTags,topics,targetAudience" if include_content else "docId,docName,fileName,summary,documentType,industry,company,reportYear,author,pageCount,uploadDate"
            )
            
        elif search_type == "text":
            # Text search only
            search_results = search_client.search(
                search_text=query,
                filter=filter_expression,
                top=top_k,
                select="docId,docName,fileName,content,summary,documentType,industry,company,reportYear,author,pageCount,uploadDate,keywords,documentTags,topics,targetAudience" if include_content else "docId,docName,fileName,summary,documentType,industry,company,reportYear,author,pageCount,uploadDate"
            )
        
        # Process results
        documents = []
        for result in search_results:
            doc = dict(result)
            
            # Add search metadata
            if hasattr(result, '@search.score'):
                doc['search_score'] = result['@search.score']
            if hasattr(result, '@search.reranker_score'):
                doc['reranker_score'] = result['@search.reranker_score']
            if hasattr(result, '@search.captions'):
                doc['captions'] = [caption.text for caption in result['@search.captions']]
            
            # Truncate content if needed for response size
            if include_content and 'content' in doc and len(doc['content']) > 2000:
                doc['content_preview'] = doc['content'][:2000] + "..."
                if not include_content:
                    del doc['content']
            
            documents.append(doc)
        
        # Get search answers if available
        answers = []
        if hasattr(search_results, 'get_answers') and search_results.get_answers():
            answers = [{"text": answer.text, "score": answer.score} for answer in search_results.get_answers()]
        
        return {
            "status": "success",
            "query": query,
            "search_type": search_type,
            "filter": filter_expression,
            "total_results": len(documents),
            "documents": documents,
            "answers": answers,
            "message": f"Found {len(documents)} documents for query: {query}"
        }
        
    except Exception as e:
        print(f"❌ Error searching documents: {str(e)}")
        return {
            "status": "error",
            "message": f"Error searching documents: {str(e)}"
        }


def get_document_by_id(doc_id: str) -> Dict[str, Any]:
    """
    Retrieve a specific document by its ID.
    
    Args:
        doc_id: Document ID
    
    Returns:
        Dict containing document data or error message
    """
    try:
        document = search_client.get_document(key=doc_id)
        return {
            "status": "success",
            "document": dict(document)
        }
    except Exception as e:
        return {
            "status": "error",
            "message": f"Document not found: {str(e)}"
        }


def list_documents(
    document_type: Optional[str] = None,
    industry: Optional[str] = None,
    company: Optional[str] = None,
    top: int = 20
) -> Dict[str, Any]:
    """
    List documents with optional filtering.
    
    Args:
        document_type: Filter by document type
        industry: Filter by industry
        company: Filter by company
        top: Number of documents to return
    
    Returns:
        Dict containing list of documents
    """
    try:
        # Build filter
        filter_parts = []
        if document_type:
            filter_parts.append(f"documentType eq '{document_type}'")
        if industry:
            filter_parts.append(f"industry eq '{industry}'")
        if company:
            filter_parts.append(f"company eq '{company}'")
        
        filter_expression = " and ".join(filter_parts) if filter_parts else None
        
        results = search_client.search(
            search_text="*",
            filter=filter_expression,
            top=top,
            select="docId,docName,fileName,documentType,industry,company,reportYear,author,pageCount,uploadDate,summary",
            order_by=["uploadDate desc"]
        )
        
        documents = [dict(result) for result in results]
        
        return {
            "status": "success",
            "total_results": len(documents),
            "documents": documents
        }
        
    except Exception as e:
        return {
            "status": "error",
            "message": f"Error listing documents: {str(e)}"
        }

### Usage Examples and Testing

Examples of how to use the upload_doc_plugin 

In [13]:
# Example 1: Upload a PDF document
UPLOAD_TEST_DOCUMENT = True

if UPLOAD_TEST_DOCUMENT:
    # Example PDF file path (replace with your actual PDF file path)
    pdf_file_path = "IR_251Q_CASA.pdf"
    
    # Upload document with metadata
    upload_result = upload_doc_plugin(
        file_path=pdf_file_path,
        doc_name="Q1 2025 CASA Report",
        document_type="IR_REPORT",
        industry="Technology",
        company="Credit Agricole Group",
        report_year="2025",
        author="IR Team"
    )
    
    print("Upload Result:")
    print(json.dumps(upload_result, indent=2, ensure_ascii=False))
else:
    print("Document upload test is disabled. Set UPLOAD_TEST_DOCUMENT = True to test.")

Processing document: IR_251Q_CASA.pdf
✅ Document IR_251Q_CASA.pdf uploaded successfully!
Upload Result:
{
  "status": "success",
  "doc_id": "f27c9d12fe31888d6fe283ec21ac696c",
  "doc_name": "Q1 2025 CASA Report",
  "message": "Document IR_251Q_CASA.pdf processed and uploaded successfully",
  "page_count": 78,
  "content_length": 156356
}


### Search documents using different methods

Examples of how to use the ai_search_plugin functions.

In [ ]:
# 
SEARCH_TEST = True

if SEARCH_TEST:
    print("=== Testing Document Search ===\n")
    
    # Test 1: Hybrid search
    print("1. Hybrid Search:")
    hybrid_results = ai_search_plugin(
        query="Credit Agricole Group 2025",
        search_type="hybrid",
        top_k=3,
        include_content=True
    )
    print(f"Status: {hybrid_results['status']}")
    print(f"Found {hybrid_results.get('total_results', 0)} documents")
    if hybrid_results['status'] == 'success':
        for i, doc in enumerate(hybrid_results['documents'][:2], 1):
            print(f"  {i}. {doc.get('docName', 'N/A')} ({doc.get('company', 'N/A')})")
            print(" ==============📄content[:100]===============")
            print(f"{doc.get('content', 'N/A')[:100]}")
    print()
    
    # Test 2: Semantic search with filters
    print("2. Semantic Search with Filters:")
    semantic_results = ai_search_plugin(
        query="revenue growth and market analysis",
        search_type="semantic",
        document_type="IR_REPORT",
        company="Credit Agricole Group",
        top_k=3,
        include_content=True
    )
    print(f"Status: {semantic_results['status']}")
    print(f"Found {semantic_results.get('total_results', 0)} documents")
    if semantic_results['status'] == 'success':
        for i, doc in enumerate(semantic_results['documents'][:2], 1):
            print(f"  {i}. {doc.get('docName', 'N/A')} - {doc.get('documentType', 'N/A')}")
            print(" ==============📄content[:100]===============")
            print(f"{doc.get('content', 'N/A')[:100]}")
    print()
    
    # Test 3: Vector search
    print("3. Vector Search:")
    vector_results = ai_search_plugin(
        query="financial metrics and KPI analysis",
        search_type="vector",
        top_k=3,
        include_content=True
    )
    print(f"Status: {vector_results['status']}")
    print(f"Found {vector_results.get('total_results', 0)} documents")
    if vector_results['status'] == 'success':
        for i, doc in enumerate(vector_results['documents'][:2], 1):
            print(f"  {i}. {doc.get('docName', 'N/A')} (Score: {doc.get('search_score', 'N/A')})")
            print(" ==============📄content[:100]===============")
            print(f"{doc.get('content', 'N/A')[:100]}")
    print()
    
    # Test 4: List all documents
    print("4. List All Documents:")
    list_results = list_documents(top=5)
    print(f"Status: {list_results['status']}")
    print(f"Total documents: {list_results.get('total_results', 0)}")
    if list_results['status'] == 'success':
        for i, doc in enumerate(list_results['documents'][:3], 1):
            print(f"  {i}. {doc.get('docName', 'N/A')} - {doc.get('uploadDate', 'N/A')}")
    
else:
    print("Search test is disabled. Set SEARCH_TEST = True to test.")

=== Testing Document Search ===

1. Hybrid Search:


Status: success
Found 1 documents
  1. Q1 2025 CASA Report (Credit Agricole Group)
 ==============📄content[:100]===============
)01(


<figure>
</figure>


<figure>
</figure>


RESULTS
1ST QUARTER 2025

WORKING EVERY DAY IN THE 

2. Semantic Search with Filters:
Status: success
Found 1 documents
  1. Q1 2025 CASA Report - IR_REPORT
 ==============📄content[:100]===============
)01(


<figure>
</figure>


<figure>
</figure>


RESULTS
1ST QUARTER 2025

WORKING EVERY DAY IN THE 

3. Vector Search:
Status: success
Found 1 documents
  1. Q1 2025 CASA Report (Score: N/A)
 ==============📄content[:100]===============
)01(


<figure>
</figure>


<figure>
</figure>


RESULTS
1ST QUARTER 2025

WORKING EVERY DAY IN THE 

4. List All Documents:
Status: success
Total documents: 1
  1. Q1 2025 CASA Report - 2025-09-09T13:07:29.846Z
 ==============📄content[:100]===============
N/A
